<a href="https://colab.research.google.com/github/kangelaa/pic16b-final-proj/blob/main/data_acquisition_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install textblob
!pip install spotipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.3/240.3 kB 16.9 MB/s eta 0:00:00


In [3]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
import spotipy.oauth2 as oauth2
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from textblob import TextBlob
import re
import json
import csv
import time
from spotipy.exceptions import SpotifyException
from requests.exceptions import ReadTimeout
import numpy as np


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Part 2: Obtaining data features from Spotify API

In [5]:
rawdf = pd.read_csv("/content/drive/Shareddrives/pic16b_final/Data/playlists_250_cleaned.csv")

In [6]:
rawdf

,playlist_name,track_name,artist_name,album_name,track_uri,songartistconcat
0,Throwbacks,Lose Control (feat. Ciara & Fat Man Scoop),Missy Elliott,The Cookbook,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,Missy ElliottLose Control (feat. Ciara & Fat M...
1,Throwbacks,Toxic,Britney Spears,In The Zone,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,Britney SpearsToxic
2,Throwbacks,Crazy In Love,Beyoncé,Dangerously In Love (Alben für die Ewigkeit),spotify:track:0WqIKmW4BTrj3eJFmnCKMv,BeyoncéCrazy In Love
3,Throwbacks,Rock Your Body,Justin Timberlake,Justified,spotify:track:1AWQoqb9bSvzTjaLralEkT,Justin TimberlakeRock Your Body
4,Throwbacks,It Wasn't Me,Shaggy,Hot Shot,spotify:track:1lzr43nnXAijIGYnCT8M8H,ShaggyIt Wasn't Me
...,...,...,...,...,...,...
1090989,March 2016,Way Back Home,Cameron AG,Way Back Home,spotify:track:434Ad8AaHLADM1moFxHnYM,Cameron AGWay Back Home
1090990,March 2016,Pretty Boy Floyd,Kaleo,Kaleo,spotify:track:0ujjgOP4F3fqAX8oycq68r,KaleoPretty Boy Floyd
1090991,March 2016,Pour Sugar On Me,Kaleo,Kaleo,spotify:track:7ollvwcsbkTBbD2viii118,KaleoPour Sugar On Me
1090992,March 2016,Up in the Sky,Kaleo,Kaleo,spotify:track:2VL5Wbrn0K68R7iHgPjRqz,KaleoUp in the Sky


## Part 3: Authenticate API credentials and obtain token

In [ ]:
client_id = '44da2d1176574c93aae297d8e4956580'
client_secret = '56ff28161a914584b4915452a4f52383'
redirect_uri = 'https://localhost:3001/'

# client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
# sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)


In [ ]:
import spotipy.oauth2 as oauth2

# Set up OAuth2 authentication
auth = oauth2.SpotifyOAuth(client_id=client_id, client_secret=client_secret, redirect_uri=redirect_uri, scope='user-library-read')

# Get authorization URL
auth_url = auth.get_authorize_url()

# Redirect user to authorization URL
print("Please visit this URL to authorize the app:", auth_url)

# Prompt user to enter authorization code
code = input("Please enter the authorization code: ")

# Get access token using authorization code
token_info = auth.get_access_token(code)
access_token = token_info['access_token']

# Set up Spotify API instance
sp = spotipy.Spotify(auth=access_token)

Please visit this URL to authorize the app: https://accounts.spotify.com/authorize?client_id=44da2d1176574c93aae297d8e4956580&response_type=code&redirect_uri=https%3A%2F%2Flocalhost%3A3001%2F&scope=user-library-read
Please enter the authorization code: AQDwVvso6rj6dXRDNRPQCr9X58dZ_uTJJWnSUKR-Z08aTorIQzGAkqy2oRIf5WhjP5ch3R0x_fi8iRt11mPbqvX-rAionx0GWgHGAKEarnD2HKave_NAupF108Ocxe54QY1UiuoUkwDe_pDg-ht0S3Ix9b4FbT7gHKcvG80wLW10mjvjaHEXELd7l6M9wA


<ipython-input-7-4e68cf45dcaa>:16: DeprecationWarning: You're using 'as_dict = True'.get_access_token will return the token string directly in future versions. Please adjust your code accordingly, or use get_cached_token instead.
  token_info = auth.get_access_token(code)


In [ ]:
# Functions to refresh token in case of going over API rate limits

def create_spotify():
    auth_manager = SpotifyOAuth(
        scope='user-library-read',
        #username=USERNAME,
        redirect_uri=redirect_uri,
        client_id=client_id,
        client_secret=client_secret)

    spotify = spotipy.Spotify(auth_manager=auth_manager)

    return auth_manager, spotify


In [ ]:
def refresh_spotify(auth_manager, spotify):
    token_info = auth_manager.cache_handler.get_cached_token()
    if auth_manager.is_token_expired(token_info):
        auth_manager, spotify = create_spotify()
    return auth_manager, spotify

In [ ]:
refresh_spotify(auth, sp)

(<spotipy.oauth2.SpotifyOAuth at 0x7fcdbd5baa10>,
 <spotipy.client.Spotify at 0x7fcdf562abc0>)

## Part 4: Obtain audio features given track uri from our raw data-frame

In [ ]:
track_uris = rawdf['track_uri'].tolist()

In [ ]:
def audio_features(chunk_start, chunk_end, uri_chunks, auth, sp):
    audio_features = []

    for i, chunk in enumerate(uri_chunks):
        try:

            if (i >= chunk_start and i < chunk_end):
                auth,sp=refresh_spotify(auth,sp)
                chunk_features = sp.audio_features(chunk)
                audio_features.extend(chunk_features)
                print(i)
            else:
                break

            if (i + 1) % 1000 == 0:
                print('sleep')
                time.sleep(30)  # Delay for 30 seconds


        except SpotifyException as e:
            if e.http_status == 429:
                print("Rate limit exceeded. Retrying after 30 seconds.")
                for uri in chunk:
                    audio_features.append(None)  # Add a row of None values for the skipped track
                time.sleep(30)
                continue
            else:
                raise e
                print("Other Error")
                for uri in chunk:
                    audio_features.append(None)  # Add a row of None values for the skipped track

    audio_features = pd.DataFrame(audio_features)

    return audio_features



In [ ]:
hundred_uri_chunks = [track_uris[i:i + 100] for i in range(0, len(track_uris), 100)]
chunk_start = 0
chunk_end = len(hundred_uri_chunks)

# get audio features for first 5000 uri chunks
audio_features = audio_features(chunk_start, chunk_end, hundred_uri_chunks, auth, sp)


In [ ]:
audio_features.to_csv('audio_features_full.csv')

In [7]:
audio_features_full = pd.read_csv('/content/drive/Shareddrives/pic16b_final/Data/audio_features_full.csv')
audio_features_full

,Unnamed: 0.1,Unnamed: 0,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0,0,0.904,0.813,4.0,-7.105,0.0,0.1210,0.03110,0.006970,0.0471,0.810,125.461,audio_features,0UaMYEvWZi0ZqiDOoHU3YI,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,https://api.spotify.com/v1/tracks/0UaMYEvWZi0Z...,https://api.spotify.com/v1/audio-analysis/0UaM...,226864.0,4.0
1,1,1,0.774,0.838,5.0,-3.914,0.0,0.1140,0.02490,0.025000,0.2420,0.924,143.040,audio_features,6I9VzXrHxO9rA9A5euc8Ak,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,https://api.spotify.com/v1/tracks/6I9VzXrHxO9r...,https://api.spotify.com/v1/audio-analysis/6I9V...,198800.0,4.0
2,2,2,0.664,0.758,2.0,-6.583,0.0,0.2100,0.00238,0.000000,0.0598,0.701,99.259,audio_features,0WqIKmW4BTrj3eJFmnCKMv,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,https://api.spotify.com/v1/tracks/0WqIKmW4BTrj...,https://api.spotify.com/v1/audio-analysis/0WqI...,235933.0,4.0
3,3,3,0.892,0.714,4.0,-6.055,0.0,0.1410,0.20100,0.000234,0.0521,0.817,100.972,audio_features,1AWQoqb9bSvzTjaLralEkT,spotify:track:1AWQoqb9bSvzTjaLralEkT,https://api.spotify.com/v1/tracks/1AWQoqb9bSvz...,https://api.spotify.com/v1/audio-analysis/1AWQ...,267267.0,4.0
4,4,4,0.853,0.606,0.0,-4.596,1.0,0.0713,0.05610,0.000000,0.3130,0.654,94.759,audio_features,1lzr43nnXAijIGYnCT8M8H,spotify:track:1lzr43nnXAijIGYnCT8M8H,https://api.spotify.com/v1/tracks/1lzr43nnXAij...,https://api.spotify.com/v1/audio-analysis/1lzr...,227600.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1090961,160589,160589,0.467,0.620,9.0,-7.234,1.0,0.0520,0.36600,0.043200,0.1090,0.477,165.937,audio_features,434Ad8AaHLADM1moFxHnYM,spotify:track:434Ad8AaHLADM1moFxHnYM,https://api.spotify.com/v1/tracks/434Ad8AaHLAD...,https://api.spotify.com/v1/audio-analysis/434A...,150871.0,4.0
1090962,160590,160590,0.492,0.834,10.0,-5.121,1.0,0.0482,0.02060,0.000010,0.3670,0.686,126.477,audio_features,0ujjgOP4F3fqAX8oycq68r,spotify:track:0ujjgOP4F3fqAX8oycq68r,https://api.spotify.com/v1/tracks/0ujjgOP4F3fq...,https://api.spotify.com/v1/audio-analysis/0ujj...,227370.0,4.0
1090963,160591,160591,0.375,0.717,0.0,-5.548,1.0,0.0499,0.04020,0.000000,0.3530,0.709,169.802,audio_features,7ollvwcsbkTBbD2viii118,spotify:track:7ollvwcsbkTBbD2viii118,https://api.spotify.com/v1/tracks/7ollvwcsbkTB...,https://api.spotify.com/v1/audio-analysis/7oll...,166441.0,4.0
1090964,160592,160592,0.395,0.597,9.0,-4.542,1.0,0.0341,0.28000,0.002170,0.0794,0.156,117.806,audio_features,2VL5Wbrn0K68R7iHgPjRqz,spotify:track:2VL5Wbrn0K68R7iHgPjRqz,https://api.spotify.com/v1/tracks/2VL5Wbrn0K68...,https://api.spotify.com/v1/audio-analysis/2VL5...,276039.0,4.0


In [9]:
audio_features_full = audio_features_full.drop(['Unnamed: 0.1','Unnamed: 0'], axis = 1)
audio_features_full = audio_features_full.reset_index(drop = True)
audio_features_full = audio_features_full.drop_duplicates(subset = 'uri')
audio_features_full

,index,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0,0.904,0.813,4.0,-7.105,0.0,0.1210,0.03110,0.006970,0.0471,0.810,125.461,audio_features,0UaMYEvWZi0ZqiDOoHU3YI,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,https://api.spotify.com/v1/tracks/0UaMYEvWZi0Z...,https://api.spotify.com/v1/audio-analysis/0UaM...,226864.0,4.0
1,1,0.774,0.838,5.0,-3.914,0.0,0.1140,0.02490,0.025000,0.2420,0.924,143.040,audio_features,6I9VzXrHxO9rA9A5euc8Ak,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,https://api.spotify.com/v1/tracks/6I9VzXrHxO9r...,https://api.spotify.com/v1/audio-analysis/6I9V...,198800.0,4.0
2,2,0.664,0.758,2.0,-6.583,0.0,0.2100,0.00238,0.000000,0.0598,0.701,99.259,audio_features,0WqIKmW4BTrj3eJFmnCKMv,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,https://api.spotify.com/v1/tracks/0WqIKmW4BTrj...,https://api.spotify.com/v1/audio-analysis/0WqI...,235933.0,4.0
3,3,0.892,0.714,4.0,-6.055,0.0,0.1410,0.20100,0.000234,0.0521,0.817,100.972,audio_features,1AWQoqb9bSvzTjaLralEkT,spotify:track:1AWQoqb9bSvzTjaLralEkT,https://api.spotify.com/v1/tracks/1AWQoqb9bSvz...,https://api.spotify.com/v1/audio-analysis/1AWQ...,267267.0,4.0
4,4,0.853,0.606,0.0,-4.596,1.0,0.0713,0.05610,0.000000,0.3130,0.654,94.759,audio_features,1lzr43nnXAijIGYnCT8M8H,spotify:track:1lzr43nnXAijIGYnCT8M8H,https://api.spotify.com/v1/tracks/1lzr43nnXAij...,https://api.spotify.com/v1/audio-analysis/1lzr...,227600.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1090961,1090961,0.467,0.620,9.0,-7.234,1.0,0.0520,0.36600,0.043200,0.1090,0.477,165.937,audio_features,434Ad8AaHLADM1moFxHnYM,spotify:track:434Ad8AaHLADM1moFxHnYM,https://api.spotify.com/v1/tracks/434Ad8AaHLAD...,https://api.spotify.com/v1/audio-analysis/434A...,150871.0,4.0
1090962,1090962,0.492,0.834,10.0,-5.121,1.0,0.0482,0.02060,0.000010,0.3670,0.686,126.477,audio_features,0ujjgOP4F3fqAX8oycq68r,spotify:track:0ujjgOP4F3fqAX8oycq68r,https://api.spotify.com/v1/tracks/0ujjgOP4F3fq...,https://api.spotify.com/v1/audio-analysis/0ujj...,227370.0,4.0
1090963,1090963,0.375,0.717,0.0,-5.548,1.0,0.0499,0.04020,0.000000,0.3530,0.709,169.802,audio_features,7ollvwcsbkTBbD2viii118,spotify:track:7ollvwcsbkTBbD2viii118,https://api.spotify.com/v1/tracks/7ollvwcsbkTB...,https://api.spotify.com/v1/audio-analysis/7oll...,166441.0,4.0
1090964,1090964,0.395,0.597,9.0,-4.542,1.0,0.0341,0.28000,0.002170,0.0794,0.156,117.806,audio_features,2VL5Wbrn0K68R7iHgPjRqz,spotify:track:2VL5Wbrn0K68R7iHgPjRqz,https://api.spotify.com/v1/tracks/2VL5Wbrn0K68...,https://api.spotify.com/v1/audio-analysis/2VL5...,276039.0,4.0


## Part 5: Obtain track/artist popularity based on track uri

In [ ]:
def track_features(chunk_start, chunk_end, uri_chunks, auth, sp):
  track_info = []

  for i, chunk in enumerate(uri_chunks):
        try:
            if (i >= chunk_start and i < chunk_end):
                auth,sp=refresh_spotify(auth,sp)
                chunk_tracks = sp.tracks(chunk)['tracks']
                track_info.extend(chunk_tracks)
                print(i)
            else:
                break

            if (i + 1) % 1000 == 0:
                print('sleep')
                time.sleep(30)  # Delay for 30 seconds


        except SpotifyException as e:
            if e.http_status == 429:
                print("Rate limit exceeded. Retrying after 30 seconds.")
                for uri in chunk:
                    track_info.append(None)  # Add a row of None values for the skipped track
                time.sleep(30)
                continue
            else:
                raise e
                print("Other Error")
                for uri in chunk:
                    track_info.append(None)  # Add a row of None values for the skipped track

  track_info = pd.DataFrame(track_info)

  return track_info

In [ ]:
fifty_uri_chunks = [track_uris[i:i + 50] for i in range(0, len(track_uris), 50)]
chunk_start = 0
chunk_end = len(fifty_uri_chunks)

track_features = track_features(chunk_start, chunk_end, fifty_uri_chunks, auth, sp)


In [ ]:
track_features.to_csv('tracks_features_full.csv')

In [11]:
track_features_full = pd.read_csv('/content/drive/Shareddrives/pic16b_final/Data/track_features_full.csv')
track_features_full

<ipython-input-11-5ecad454c37a>:1: DtypeWarning: Columns (7,12) have mixed types. Specify dtype option on import or set low_memory=False.
  track_features_full = pd.read_csv('/content/drive/Shareddrives/pic16b_final/Data/track_features_full.csv')


,Unnamed: 0.1,Unnamed: 0,album,artists,available_markets,disc_number,duration_ms,explicit,external_ids,external_urls,href,id,is_local,name,popularity,preview_url,track_number,type,uri
0,0,0,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",1.0,226863.0,True,{'isrc': 'USEE10414022'},{'spotify': 'https://open.spotify.com/track/0U...,https://api.spotify.com/v1/tracks/0UaMYEvWZi0Z...,0UaMYEvWZi0ZqiDOoHU3YI,False,Lose Control (feat. Ciara & Fat Man Scoop),69.0,https://p.scdn.co/mp3-preview/253a76c453026570...,4.0,track,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI
1,1,1,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",1.0,198800.0,False,{'isrc': 'USJI10301005'},{'spotify': 'https://open.spotify.com/track/6I...,https://api.spotify.com/v1/tracks/6I9VzXrHxO9r...,6I9VzXrHxO9rA9A5euc8Ak,False,Toxic,84.0,https://p.scdn.co/mp3-preview/6de2791f84c1d637...,6.0,track,spotify:track:6I9VzXrHxO9rA9A5euc8Ak
2,2,2,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"['CH', 'LI']",1.0,235933.0,False,{'isrc': 'USSM10305425'},{'spotify': 'https://open.spotify.com/track/0W...,https://api.spotify.com/v1/tracks/0WqIKmW4BTrj...,0WqIKmW4BTrj3eJFmnCKMv,False,Crazy In Love (feat. Jay-Z),21.0,https://p.scdn.co/mp3-preview/023ccc9c406cafc5...,1.0,track,spotify:track:0WqIKmW4BTrj3eJFmnCKMv
3,3,3,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",1.0,267266.0,False,{'isrc': 'USJI10200367'},{'spotify': 'https://open.spotify.com/track/1A...,https://api.spotify.com/v1/tracks/1AWQoqb9bSvz...,1AWQoqb9bSvzTjaLralEkT,False,Rock Your Body,79.0,https://p.scdn.co/mp3-preview/a4cdd210220f23d3...,6.0,track,spotify:track:1AWQoqb9bSvzTjaLralEkT
4,4,4,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,[],1.0,227600.0,False,{'isrc': 'USMC10000393'},{'spotify': 'https://open.spotify.com/track/1l...,https://api.spotify.com/v1/tracks/1lzr43nnXAij...,1lzr43nnXAijIGYnCT8M8H,False,It Wasn't Me,4.0,NaN,3.0,track,spotify:track:1lzr43nnXAijIGYnCT8M8H
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1091089,90989,90989,"{'album_type': 'single', 'artists': [{'externa...",[{'external_urls': {'spotify': 'https://open.s...,[],1.0,150870.0,False,{'isrc': 'GBKPL1663292'},{'spotify': 'https://open.spotify.com/track/43...,https://api.spotify.com/v1/tracks/434Ad8AaHLAD...,434Ad8AaHLADM1moFxHnYM,False,Way Back Home,0.0,NaN,1.0,track,spotify:track:434Ad8AaHLADM1moFxHnYM
1091090,90990,90990,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,[],1.0,227369.0,False,{'isrc': 'ISA111303806'},{'spotify': 'https://open.spotify.com/track/0u...,https://api.spotify.com/v1/tracks/0ujjgOP4F3fq...,0ujjgOP4F3fqAX8oycq68r,False,Pretty Boy Floyd,0.0,NaN,6.0,track,spotify:track:0ujjgOP4F3fqAX8oycq68r
1091091,90991,90991,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,[],1.0,166440.0,False,{'isrc': 'ISA111303807'},{'spotify': 'https://open.spotify.com/track/7o...,https://api.spotify.com/v1/tracks/7ollvwcsbkTB...,7ollvwcsbkTBbD2viii118,False,Pour Sugar On Me,0.0,NaN,7.0,track,spotify:track:7ollvwcsbkTBbD2viii118
1091092,90992,90992,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,[],1.0,276038.0,False,{'isrc': 'ISA111303809'},{'spotify': 'https://open.spotify.com/track/2V...,https://api.spotify.com/v1/tracks/2VL5Wbrn0K68...,2VL5Wbrn0K68R7iHgPjRqz,False,Up in the Sky,0.0,NaN,9.0,track,spotify:track:2VL5Wbrn0K68R7iHgPjRqz


In [12]:
track_features_full = track_features_full.drop(['Unnamed: 0.1','Unnamed: 0'], axis = 1)
track_features_full = track_features_full.reset_index(drop = True)
track_features_full = track_features_full.drop_duplicates(subset = 'uri')
track_features_full


,index,album,artists,available_markets,disc_number,duration_ms,explicit,external_ids,external_urls,href,id,is_local,name,popularity,preview_url,track_number,type,uri
0,0,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",1.0,226863.0,True,{'isrc': 'USEE10414022'},{'spotify': 'https://open.spotify.com/track/0U...,https://api.spotify.com/v1/tracks/0UaMYEvWZi0Z...,0UaMYEvWZi0ZqiDOoHU3YI,False,Lose Control (feat. Ciara & Fat Man Scoop),69.0,https://p.scdn.co/mp3-preview/253a76c453026570...,4.0,track,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI
1,1,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",1.0,198800.0,False,{'isrc': 'USJI10301005'},{'spotify': 'https://open.spotify.com/track/6I...,https://api.spotify.com/v1/tracks/6I9VzXrHxO9r...,6I9VzXrHxO9rA9A5euc8Ak,False,Toxic,84.0,https://p.scdn.co/mp3-preview/6de2791f84c1d637...,6.0,track,spotify:track:6I9VzXrHxO9rA9A5euc8Ak
2,2,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"['CH', 'LI']",1.0,235933.0,False,{'isrc': 'USSM10305425'},{'spotify': 'https://open.spotify.com/track/0W...,https://api.spotify.com/v1/tracks/0WqIKmW4BTrj...,0WqIKmW4BTrj3eJFmnCKMv,False,Crazy In Love (feat. Jay-Z),21.0,https://p.scdn.co/mp3-preview/023ccc9c406cafc5...,1.0,track,spotify:track:0WqIKmW4BTrj3eJFmnCKMv
3,3,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",1.0,267266.0,False,{'isrc': 'USJI10200367'},{'spotify': 'https://open.spotify.com/track/1A...,https://api.spotify.com/v1/tracks/1AWQoqb9bSvz...,1AWQoqb9bSvzTjaLralEkT,False,Rock Your Body,79.0,https://p.scdn.co/mp3-preview/a4cdd210220f23d3...,6.0,track,spotify:track:1AWQoqb9bSvzTjaLralEkT
4,4,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,[],1.0,227600.0,False,{'isrc': 'USMC10000393'},{'spotify': 'https://open.spotify.com/track/1l...,https://api.spotify.com/v1/tracks/1lzr43nnXAij...,1lzr43nnXAijIGYnCT8M8H,False,It Wasn't Me,4.0,NaN,3.0,track,spotify:track:1lzr43nnXAijIGYnCT8M8H
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1091089,1091089,"{'album_type': 'single', 'artists': [{'externa...",[{'external_urls': {'spotify': 'https://open.s...,[],1.0,150870.0,False,{'isrc': 'GBKPL1663292'},{'spotify': 'https://open.spotify.com/track/43...,https://api.spotify.com/v1/tracks/434Ad8AaHLAD...,434Ad8AaHLADM1moFxHnYM,False,Way Back Home,0.0,NaN,1.0,track,spotify:track:434Ad8AaHLADM1moFxHnYM
1091090,1091090,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,[],1.0,227369.0,False,{'isrc': 'ISA111303806'},{'spotify': 'https://open.spotify.com/track/0u...,https://api.spotify.com/v1/tracks/0ujjgOP4F3fq...,0ujjgOP4F3fqAX8oycq68r,False,Pretty Boy Floyd,0.0,NaN,6.0,track,spotify:track:0ujjgOP4F3fqAX8oycq68r
1091091,1091091,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,[],1.0,166440.0,False,{'isrc': 'ISA111303807'},{'spotify': 'https://open.spotify.com/track/7o...,https://api.spotify.com/v1/tracks/7ollvwcsbkTB...,7ollvwcsbkTBbD2viii118,False,Pour Sugar On Me,0.0,NaN,7.0,track,spotify:track:7ollvwcsbkTBbD2viii118
1091092,1091092,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,[],1.0,276038.0,False,{'isrc': 'ISA111303809'},{'spotify': 'https://open.spotify.com/track/2V...,https://api.spotify.com/v1/tracks/2VL5Wbrn0K68...,2VL5Wbrn0K68R7iHgPjRqz,False,Up in the Sky,0.0,NaN,9.0,track,spotify:track:2VL5Wbrn0K68R7iHgPjRqz


## Part 6: Merge track/audio features with rawdf based on track uri



In [13]:
# First we merge track features df with rawdf

track_rawdf_merge = pd.merge(track_features_full, rawdf, left_on='uri',
                             right_on = 'track_uri', how='inner')

In [14]:
track_rawdf_merge.shape

(1090601, 24)

In [15]:
# Now, we merge on audio features df with the merged df from before

rawdf_trackft_audioft_merge = pd.merge(track_rawdf_merge, audio_features_full,
                                       left_on = 'uri',
                                       right_on = 'uri', how = 'inner')

In [16]:
rawdf_trackft_audioft_merge.shape

(1090573, 42)

In [17]:
merged_df = rawdf_trackft_audioft_merge

## Part 7: Given that we have a column with track URI, we will find the artist URI so we can merge on artist data that we will obtain later

In [ ]:
def artist_uri(chunk_start, chunk_end, uri_chunks, auth, sp):


  artist_uris = []  # List to store artist URIs
  track_uri_given_artist = [] # List to store track URIs

  for i, chunk in enumerate(uri_chunks):
    try:
        if (i >= chunk_start and i < chunk_end):
          auth, sp = refresh_spotify(auth, sp)
          chunk_tracks = sp.tracks(chunk)['tracks']

        # Extract artist URIs from the track information and append to the artist_uris list
          for track in chunk_tracks:
              if track == None:
                  continue
              temp_track_uri = track['uri']
              artist_uri = track['artists'][0]['uri']
              artist_uris.append(artist_uri)
              track_uri_given_artist.append(temp_track_uri)
        else:
          break

        print(i)

        # Check if the current chunk is a multiple of 1000
        if (i + 1) % 1000 == 0:
            print('sleep')
            time.sleep(30)  # Delay for 30 seconds

    except SpotifyException as e:
        if e.http_status == 429:
            print("Rate limit exceeded. Retrying after 30 seconds.")
            for uri in chunk:
                artist_uris.append(None)
                track_uri_given_artist.append(None)
            time.sleep(30)
            continue
        else:
            raise e
            print("Other Error")
            for uri in chunk:
                artist_uris.append(None)
                track_uri_given_artist.append(None)

  return artist_uris, track_uri_given_artist


In [ ]:
track_uris_col = merged_df['uri'].tolist()

In [ ]:
fifty_uri_chunks = [track_uris_col[i:i + 50] for i in range(0, len(track_uris_col), 50)]
chunk_start = 0
chunk_end = len(fifty_uri_chunks)

artist_link, track_link = artist_uri(chunk_start, chunk_end, fifty_uri_chunks, auth, sp)
pd.DataFrame({'artist_uri': artist_link, 'track_uri': track_link}).to_csv("artist_track_uri_full.csv")

In [22]:
artist_track_uri_df = pd.read_csv('/content/drive/Shareddrives/pic16b_final/Data/artist_track_uri_full.csv')
artist_track_uri_df

,Unnamed: 0.1,Unnamed: 0,artist_uri,track_uri
0,0,0,spotify:artist:2wIVse2owClT7go1WT98tk,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI
1,1,1,spotify:artist:26dSoYclwsYLMAKD3tpOr4,spotify:track:6I9VzXrHxO9rA9A5euc8Ak
2,2,2,spotify:artist:6vWDO969PvNqNYHIOW5v0m,spotify:track:0WqIKmW4BTrj3eJFmnCKMv
3,3,3,spotify:artist:31TPClRtHm23RisEBtV3X7,spotify:track:1AWQoqb9bSvzTjaLralEkT
4,4,4,spotify:artist:5EvFsr3kj42KNv97ZEnqij,spotify:track:1lzr43nnXAijIGYnCT8M8H
...,...,...,...,...
1090268,91368,91368,spotify:artist:1JOiO3DL7byBZMLDPvCDCx,spotify:track:434Ad8AaHLADM1moFxHnYM
1090269,91369,91369,spotify:artist:7jdFEYD2LTYjfwxOdlVjmc,spotify:track:0ujjgOP4F3fqAX8oycq68r
1090270,91370,91370,spotify:artist:7jdFEYD2LTYjfwxOdlVjmc,spotify:track:7ollvwcsbkTBbD2viii118
1090271,91371,91371,spotify:artist:7jdFEYD2LTYjfwxOdlVjmc,spotify:track:2VL5Wbrn0K68R7iHgPjRqz


In [27]:
artist_track_uri_df = artist_track_uri_df.drop(['Unnamed: 0.1','Unnamed: 0'], axis = 1)
artist_track_uri_df = artist_track_uri_df.reset_index(drop = True)
artist_track_uri_df = artist_track_uri_df.drop_duplicates(subset = ['artist_uri', 'track_uri'], keep = 'first')

In [28]:
artist_track_uri_df

,artist_uri,track_uri
0,spotify:artist:2wIVse2owClT7go1WT98tk,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI
1,spotify:artist:26dSoYclwsYLMAKD3tpOr4,spotify:track:6I9VzXrHxO9rA9A5euc8Ak
2,spotify:artist:6vWDO969PvNqNYHIOW5v0m,spotify:track:0WqIKmW4BTrj3eJFmnCKMv
3,spotify:artist:31TPClRtHm23RisEBtV3X7,spotify:track:1AWQoqb9bSvzTjaLralEkT
4,spotify:artist:5EvFsr3kj42KNv97ZEnqij,spotify:track:1lzr43nnXAijIGYnCT8M8H
...,...,...
1090218,spotify:artist:1JOiO3DL7byBZMLDPvCDCx,spotify:track:434Ad8AaHLADM1moFxHnYM
1090219,spotify:artist:7jdFEYD2LTYjfwxOdlVjmc,spotify:track:0ujjgOP4F3fqAX8oycq68r
1090220,spotify:artist:7jdFEYD2LTYjfwxOdlVjmc,spotify:track:7ollvwcsbkTBbD2viii118
1090221,spotify:artist:7jdFEYD2LTYjfwxOdlVjmc,spotify:track:2VL5Wbrn0K68R7iHgPjRqz


## Part 8: Merge artist/track uri csv to total dataframe to have artist uri on dataframe for merging the artist features

In [29]:
rawdf_trackft_audioft_artisttrackuri_merge = pd.merge(merged_df, artist_track_uri_df,
                                                 left_on = 'uri', right_on = 'track_uri', how = 'inner')

rawdf_trackft_audioft_artisttrackuri_merge

,index_x,album,artists,available_markets,disc_number,duration_ms_x,explicit,external_ids,external_urls,href,...,valence,tempo,type_y,id_y,track_href,analysis_url,duration_ms_y,time_signature,artist_uri,track_uri_y
0,0,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",1.0,226863.0,True,{'isrc': 'USEE10414022'},{'spotify': 'https://open.spotify.com/track/0U...,https://api.spotify.com/v1/tracks/0UaMYEvWZi0Z...,...,0.810,125.461,audio_features,0UaMYEvWZi0ZqiDOoHU3YI,https://api.spotify.com/v1/tracks/0UaMYEvWZi0Z...,https://api.spotify.com/v1/audio-analysis/0UaM...,226864.0,4.0,spotify:artist:2wIVse2owClT7go1WT98tk,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI
1,1,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",1.0,198800.0,False,{'isrc': 'USJI10301005'},{'spotify': 'https://open.spotify.com/track/6I...,https://api.spotify.com/v1/tracks/6I9VzXrHxO9r...,...,0.924,143.040,audio_features,6I9VzXrHxO9rA9A5euc8Ak,https://api.spotify.com/v1/tracks/6I9VzXrHxO9r...,https://api.spotify.com/v1/audio-analysis/6I9V...,198800.0,4.0,spotify:artist:26dSoYclwsYLMAKD3tpOr4,spotify:track:6I9VzXrHxO9rA9A5euc8Ak
2,2,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"['CH', 'LI']",1.0,235933.0,False,{'isrc': 'USSM10305425'},{'spotify': 'https://open.spotify.com/track/0W...,https://api.spotify.com/v1/tracks/0WqIKmW4BTrj...,...,0.701,99.259,audio_features,0WqIKmW4BTrj3eJFmnCKMv,https://api.spotify.com/v1/tracks/0WqIKmW4BTrj...,https://api.spotify.com/v1/audio-analysis/0WqI...,235933.0,4.0,spotify:artist:6vWDO969PvNqNYHIOW5v0m,spotify:track:0WqIKmW4BTrj3eJFmnCKMv
3,3,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",1.0,267266.0,False,{'isrc': 'USJI10200367'},{'spotify': 'https://open.spotify.com/track/1A...,https://api.spotify.com/v1/tracks/1AWQoqb9bSvz...,...,0.817,100.972,audio_features,1AWQoqb9bSvzTjaLralEkT,https://api.spotify.com/v1/tracks/1AWQoqb9bSvz...,https://api.spotify.com/v1/audio-analysis/1AWQ...,267267.0,4.0,spotify:artist:31TPClRtHm23RisEBtV3X7,spotify:track:1AWQoqb9bSvzTjaLralEkT
4,4,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,[],1.0,227600.0,False,{'isrc': 'USMC10000393'},{'spotify': 'https://open.spotify.com/track/1l...,https://api.spotify.com/v1/tracks/1lzr43nnXAij...,...,0.654,94.759,audio_features,1lzr43nnXAijIGYnCT8M8H,https://api.spotify.com/v1/tracks/1lzr43nnXAij...,https://api.spotify.com/v1/audio-analysis/1lzr...,227600.0,4.0,spotify:artist:5EvFsr3kj42KNv97ZEnqij,spotify:track:1lzr43nnXAijIGYnCT8M8H
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1090218,1091089,"{'album_type': 'single', 'artists': [{'externa...",[{'external_urls': {'spotify': 'https://open.s...,[],1.0,150870.0,False,{'isrc': 'GBKPL1663292'},{'spotify': 'https://open.spotify.com/track/43...,https://api.spotify.com/v1/tracks/434Ad8AaHLAD...,...,0.477,165.937,audio_features,434Ad8AaHLADM1moFxHnYM,https://api.spotify.com/v1/tracks/434Ad8AaHLAD...,https://api.spotify.com/v1/audio-analysis/434A...,150871.0,4.0,spotify:artist:1JOiO3DL7byBZMLDPvCDCx,spotify:track:434Ad8AaHLADM1moFxHnYM
1090219,1091090,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,[],1.0,227369.0,False,{'isrc': 'ISA111303806'},{'spotify': 'https://open.spotify.com/track/0u...,https://api.spotify.com/v1/tracks/0ujjgOP4F3fq...,...,0.686,126.477,audio_features,0ujjgOP4F3fqAX8oycq68r,https://api.spotify.com/v1/tracks/0ujjgOP4F3fq...,https://api.spotify.com/v1/audio-analysis/0ujj...,227370.0,4.0,spotify:artist:7jdFEYD2LTYjfwxOdlVjmc,spotify:track:0ujjgOP4F3fqAX8oycq68r
1090220,1091091,"{'album_type': 'album'

In [30]:
rawdf_trackft_audioft_artisttrackuri_merge.shape

(1090223, 44)

### Part 9: Obtain artist popularity and genres

In [ ]:

# get artists id's for artist popularity and genres
artists = []
artists = artists.append(track_features["artists"])
artist_ids = []

for i, artist in enumerate(artists):
    if type(artist) != float:
        index=artist.find("'id': '")
        id=artist[index+7:index+29]
        artist_ids.append(id)

pd.DataFrame(artist_ids).to_csv("artist_ids.csv")

In [18]:
artist_ids = pd.read_csv("/content/drive/MyDrive/artist_ids.csv")

In [19]:
artist_ids

,Unnamed: 0,0
0,0,2wIVse2owClT7go1WT98tk
1,1,26dSoYclwsYLMAKD3tpOr4
2,2,6vWDO969PvNqNYHIOW5v0m
3,3,31TPClRtHm23RisEBtV3X7
4,4,5EvFsr3kj42KNv97ZEnqij
...,...,...
1090597,1090597,1JOiO3DL7byBZMLDPvCDCx
1090598,1090598,7jdFEYD2LTYjfwxOdlVjmc
1090599,1090599,7jdFEYD2LTYjfwxOdlVjmc
1090600,1090600,7jdFEYD2LTYjfwxOdlVjmc


In [ ]:
def artist_info(chunk_start, chunk_end, uri_chunks, auth, sp):

  artist_genres = []
  artist_popularity = []

  for i, chunk in enumerate(uri_chunks):
    try:
        if (i >= chunk_start and i < chunk_end):
          auth, sp = refresh_spotify(auth, sp)

          artist_info=sp.artists(chunk)
          artist_info=pd.DataFrame(artist_info["artists"])
          artist_popularity.extend(artist_info["popularity"])
          artist_genres.extend(artist_info["genres"])


        else:
          break

        print(i)

        # Check if the current chunk is a multiple of 1000
        if (i + 1) % 1000 == 0:
            print('sleep')
            time.sleep(30)  # Delay for 30 seconds

    except SpotifyException as e:
        if e.http_status == 429:
            print("Rate limit exceeded. Retrying after 30 seconds.")
            for uri in chunk:
                artist_uris.append(None)
                track_uri_given_artist.append(None)
            time.sleep(30)
            continue
        else:
            raise e
            print("Other Error")
            for uri in chunk:
                artist_uris.append(None)
                track_uri_given_artist.append(None)

  return artist_genres, artist_popularity

In [ ]:
artist_uri = artist_ids['0'].tolist()

In [ ]:
fifty_uri_chunks = [artist_uri[i:i + 50] for i in range(0, len(artist_uri), 50)]
chunk_start = 0
chunk_end = len(fifty_uri_chunks)

artist_genres, artist_popularity = artist_info(chunk_start, chunk_end, fifty_uri_chunks, auth, sp)
artist_uri = pd.DataFrame(artist_ids['0'].tolist(), columns = ['artist_uri'])
artist_uri['artist_uri'] = artist_uri['artist_uri'].apply(lambda x: 'spotify:artist:' + x)
artist_uri = artist_uri['artist_uri'].tolist()
pd.DataFrame({'genres': artist_genres, 'popularity': artist_popularity, 'uri': artist_ids}).to_csv("artist_ft_full.csv")

In [31]:
artist_ft_full = pd.read_csv('/content/drive/Shareddrives/pic16b_final/Data/artist_ft_full.csv')
artist_ft_full

,Unnamed: 0,genres,popularity,uri
0,0,"['dance pop', 'hip hop', 'hip pop', 'neo soul'...",72,spotify:artist:2wIVse2owClT7go1WT98tk
1,1,"['dance pop', 'pop']",80,spotify:artist:26dSoYclwsYLMAKD3tpOr4
2,2,"['pop', 'r&b']",87,spotify:artist:6vWDO969PvNqNYHIOW5v0m
3,3,"['dance pop', 'pop']",79,spotify:artist:31TPClRtHm23RisEBtV3X7
4,4,"['dance pop', 'pop rap', 'reggae fusion']",73,spotify:artist:5EvFsr3kj42KNv97ZEnqij
...,...,...,...,...
1090597,90597,['oxford indie'],6,spotify:artist:1JOiO3DL7byBZMLDPvCDCx
1090598,90598,"['icelandic indie', 'icelandic rock', 'modern ...",68,spotify:artist:7jdFEYD2LTYjfwxOdlVjmc
1090599,90599,"['icelandic indie', 'icelandic rock', 'modern ...",68,spotify:artist:7jdFEYD2LTYjfwxOdlVjmc
1090600,90600,"['icelandic indie', 'icelandic rock', 'modern ...",68,spotify:artist:7jdFEYD2LTYjfwxOdlVjmc


In [32]:
artist_ft_full = artist_ft_full.rename(columns={'popularity': 'artist_popularity', 'uri': 'artist_uri'})
artist_ft_full = artist_ft_full.drop(['Unnamed: 0'], axis = 1)
artist_ft_full = artist_ft_full.reset_index(drop = True)
artist_ft_full = artist_ft_full.drop_duplicates(subset = 'artist_uri')
artist_ft_full

,genres,artist_popularity,artist_uri
0,"['dance pop', 'hip hop', 'hip pop', 'neo soul'...",72,spotify:artist:2wIVse2owClT7go1WT98tk
1,"['dance pop', 'pop']",80,spotify:artist:26dSoYclwsYLMAKD3tpOr4
2,"['pop', 'r&b']",87,spotify:artist:6vWDO969PvNqNYHIOW5v0m
3,"['dance pop', 'pop']",79,spotify:artist:31TPClRtHm23RisEBtV3X7
4,"['dance pop', 'pop rap', 'reggae fusion']",73,spotify:artist:5EvFsr3kj42KNv97ZEnqij
...,...,...,...
1090486,[],1,spotify:artist:7ko6WoEsbuDFMvaj1O72Fd
1090487,['emocore'],10,spotify:artist:3KjTVFShVGJ2db3HkiBTBo
1090519,[],1,spotify:artist:5cXwgLFYT8jqQ2zUWnjtN1
1090565,[],0,spotify:artist:25gxmr5hSeEfav2Sp6RnN8


### Part 10: Merge artist feature data to merged data

In [34]:
merged_df = rawdf_trackft_audioft_artisttrackuri_merge

In [35]:
merged_df.columns

Index(['index_x', 'album', 'artists', 'available_markets', 'disc_number',
       'duration_ms_x', 'explicit', 'external_ids', 'external_urls', 'href',
       'id_x', 'is_local', 'name', 'popularity', 'preview_url', 'track_number',
       'type_x', 'uri', 'playlist_name', 'track_name', 'artist_name',
       'album_name', 'track_uri_x', 'songartistconcat', 'index_y',
       'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'type_y', 'id_y', 'track_href', 'analysis_url', 'duration_ms_y',
       'time_signature', 'artist_uri', 'track_uri_y'],
      dtype='object')

In [36]:
pre_final_df = merged_df[['playlist_name', 'track_name', 'artist_name',
                                     'album_name', 'danceability', 'energy', 'key', 'loudness',
                                    'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness',
                                    'valence', 'tempo', 'track_uri_x', 'artist_uri', 'popularity', 'time_signature']]

In [37]:
pre_final_df

,playlist_name,track_name,artist_name,album_name,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,track_uri_x,artist_uri,popularity,time_signature
0,Throwbacks,Lose Control (feat. Ciara & Fat Man Scoop),Missy Elliott,The Cookbook,0.904,0.813,4.0,-7.105,0.0,0.1210,0.03110,0.006970,0.0471,0.810,125.461,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,spotify:artist:2wIVse2owClT7go1WT98tk,69.0,4.0
1,Throwbacks,Toxic,Britney Spears,In The Zone,0.774,0.838,5.0,-3.914,0.0,0.1140,0.02490,0.025000,0.2420,0.924,143.040,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,spotify:artist:26dSoYclwsYLMAKD3tpOr4,84.0,4.0
2,Throwbacks,Crazy In Love,Beyoncé,Dangerously In Love (Alben für die Ewigkeit),0.664,0.758,2.0,-6.583,0.0,0.2100,0.00238,0.000000,0.0598,0.701,99.259,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,spotify:artist:6vWDO969PvNqNYHIOW5v0m,21.0,4.0
3,Throwbacks,Rock Your Body,Justin Timberlake,Justified,0.892,0.714,4.0,-6.055,0.0,0.1410,0.20100,0.000234,0.0521,0.817,100.972,spotify:track:1AWQoqb9bSvzTjaLralEkT,spotify:artist:31TPClRtHm23RisEBtV3X7,79.0,4.0
4,Throwbacks,It Wasn't Me,Shaggy,Hot Shot,0.853,0.606,0.0,-4.596,1.0,0.0713,0.05610,0.000000,0.3130,0.654,94.759,spotify:track:1lzr43nnXAijIGYnCT8M8H,spotify:artist:5EvFsr3kj42KNv97ZEnqij,4.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1090218,March 2016,Way Back Home,Cameron AG,Way Back Home,0.467,0.620,9.0,-7.234,1.0,0.0520,0.36600,0.043200,0.1090,0.477,165.937,spotify:track:434Ad8AaHLADM1moFxHnYM,spotify:artist:1JOiO3DL7byBZMLDPvCDCx,0.0,4.0
1090219,March 2016,Pretty Boy Floyd,Kaleo,Kaleo,0.492,0.834,10.0,-5.121,1.0,0.0482,0.02060,0.000010,0.3670,0.686,126.477,spotify:track:0ujjgOP4F3fqAX8oycq68r,spotify:artist:7jdFEYD2LTYjfwxOdlVjmc,0.0,4.0
1090220,March 2016,Pour Sugar On Me,Kaleo,Kaleo,0.375,0.717,0.0,-5.548,1.0,0.0499,0.04020,0.000000,0.3530,0.709,169.802,spotify:track:7ollvwcsbkTBbD2viii118,spotify:artist:7jdFEYD2LTYjfwxOdlVjmc,0.0,4.0
1090221,March 2016,Up in the Sky,Kaleo,Kaleo,0.395,0.597,9.0,-4.542,1.0,0.0341,0.28000,0.002170,0.0794,0.156,117.806,spotify:track:2VL5Wbrn0K68R7iHgPjRqz,spotify:artist:7jdFEYD2LTYjfwxOdlVjmc,0.0,4.0


In [38]:
new_column_names = {
    'popularity':'track_popularity',
    'track_uri_x' : 'track_uri',

}

pre_final_df = pre_final_df.rename(columns=new_column_names)
pre_final_df.columns

Index(['playlist_name', 'track_name', 'artist_name', 'album_name',
       'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'track_uri', 'artist_uri', 'track_popularity', 'time_signature'],
      dtype='object')

In [39]:
artist_ft_full

,genres,artist_popularity,artist_uri
0,"['dance pop', 'hip hop', 'hip pop', 'neo soul'...",72,spotify:artist:2wIVse2owClT7go1WT98tk
1,"['dance pop', 'pop']",80,spotify:artist:26dSoYclwsYLMAKD3tpOr4
2,"['pop', 'r&b']",87,spotify:artist:6vWDO969PvNqNYHIOW5v0m
3,"['dance pop', 'pop']",79,spotify:artist:31TPClRtHm23RisEBtV3X7
4,"['dance pop', 'pop rap', 'reggae fusion']",73,spotify:artist:5EvFsr3kj42KNv97ZEnqij
...,...,...,...
1090486,[],1,spotify:artist:7ko6WoEsbuDFMvaj1O72Fd
1090487,['emocore'],10,spotify:artist:3KjTVFShVGJ2db3HkiBTBo
1090519,[],1,spotify:artist:5cXwgLFYT8jqQ2zUWnjtN1
1090565,[],0,spotify:artist:25gxmr5hSeEfav2Sp6RnN8


In [40]:
spotify_final_df = pd.merge(pre_final_df, artist_ft_full, left_on = 'artist_uri', right_on = 'artist_uri', how = 'inner')
spotify_final_df.shape

(1090193, 21)

In [41]:
spotify_final_df = spotify_final_df.dropna()
spotify_final_df.shape

(1090176, 21)

In [42]:
spotify_final_df

,playlist_name,track_name,artist_name,album_name,danceability,energy,key,loudness,mode,speechiness,...,instrumentalness,liveness,valence,tempo,track_uri,artist_uri,track_popularity,time_signature,genres,artist_popularity
0,Throwbacks,Lose Control (feat. Ciara & Fat Man Scoop),Missy Elliott,The Cookbook,0.904,0.813,4.0,-7.105,0.0,0.1210,...,0.006970,0.0471,0.810,125.461,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,spotify:artist:2wIVse2owClT7go1WT98tk,69.0,4.0,"['dance pop', 'hip hop', 'hip pop', 'neo soul'...",72
1,~Rando~,Work It,Missy Elliott,Under Construction,0.884,0.677,1.0,-5.603,1.0,0.2830,...,0.000000,0.0732,0.584,101.868,spotify:track:3jagJCUbdqhDSPuxP8cAqF,spotify:artist:2wIVse2owClT7go1WT98tk,72.0,4.0,"['dance pop', 'hip hop', 'hip pop', 'neo soul'...",72
2,Party Rock!!,Get Ur Freak On,Missy Elliott,Respect M.E.,0.794,0.805,0.0,-6.554,1.0,0.2300,...,0.122000,0.0952,0.658,177.799,spotify:track:3XplJgPz8VjbDzbGwGgZdq,spotify:artist:2wIVse2owClT7go1WT98tk,45.0,4.0,"['dance pop', 'hip hop', 'hip pop', 'neo soul'...",72
3,4/20,One Minute Man (feat. Ludacris),Missy Elliott,Miss E...So Addictive,0.622,0.669,9.0,-8.419,1.0,0.3290,...,0.000003,0.1520,0.570,93.839,spotify:track:0jG92AlXau21qgCQRxGLic,spotify:artist:2wIVse2owClT7go1WT98tk,58.0,4.0,"['dance pop', 'hip hop', 'hip pop', 'neo soul'...",72
4,Rap/Pop,Gossip Folks (feat. Ludacris),Missy Elliott,Under Construction,0.707,0.538,0.0,-5.264,1.0,0.5050,...,0.000000,0.3170,0.439,121.732,spotify:track:75DjPjiIp2fvJDjtt41Jfs,spotify:artist:2wIVse2owClT7go1WT98tk,60.0,4.0,"['dance pop', 'hip hop', 'hip pop', 'neo soul'...",72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1090188,dance!,Feel (Moto Blanco Club Mix),Hamilton,Feel - Moto Blanco Remixes,0.760,0.756,9.0,-5.540,0.0,0.0609,...,0.001200,0.1060,0.808,128.006,spotify:track:0zW2tXaUCccEqysSJTpHFN,spotify:artist:5cXwgLFYT8jqQ2zUWnjtN1,0.0,4.0,[],1
1090189,iChill,Login,Drew Price's Bermuda Triangle,Friends and Family,0.496,0.704,1.0,-7.013,1.0,0.0315,...,0.810000,0.1550,0.327,88.008,spotify:track:5eYBX2aHB3OxSNKvCdOs9G,spotify:artist:25gxmr5hSeEfav2Sp6RnN8,0.0,4.0,[],0
1090190,iChill,Something White,Drew Price's Bermuda Triangle,Friends and Family,0.583,0.505,5.0,-7.385,1.0,0.0251,...,0.933000,0.1140,0.361,82.942,spotify:track:0DKG8yutFPENxGMCcfyJTx,spotify:artist:25gxmr5hSeEfav2Sp6RnN8,0.0,4.0,[],0
1090191,iChill,Friends and Family,Drew Price's Bermuda Triangle,Friends and Family,0.385,0.578,8.0,-5.445,0.0,0.0371,...,0.634000,0.1030,0.530,134.984,spotify:track:5T401bkZZkCeIOvvJimU5j,spotify:artist:25gxmr5hSeEfav2Sp6RnN8,0.0,4.0,[],0


In [ ]:
spotify_final_df.to_csv('spotify_final_data.csv')